In [1]:
!pip install -U scikit-learn scipy matplotlib
!pip install transformers
!pip install datasets
!pip install nlp
!pip install colorama
!pip install torch

  Using cached scikit_learn-1.3.2-cp311-cp311-win_amd64.whl (9.2 MB)
  Using cached scipy-1.11.3-cp311-cp311-win_amd64.whl (44.1 MB)
  Using cached matplotlib-3.8.0-cp311-cp311-win_amd64.whl (7.6 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.


  Using cached datasets-2.14.6-py3-none-any.whl (493 kB)
  Using cached xxhash-3.4.1-cp311-cp311-win_amd64.whl (29 kB)
  Using cached multiprocess-0.70.15-py311-none-any.whl (135 kB)
                                              0.0/302.0 kB ? eta -:--:--
     -----                                    41.0/302.0 kB ? eta -:--:--
     -----------                             92.2/302.0 kB 1.7 MB/s eta 0:00:01
     ---------------------                  174.1/302.0 kB 2.1 MB/s eta 0:00:01
     ----------------------------------     276.5/302.0 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 302.0/302.0 kB 1.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Unins

In [2]:
import torch, os, re, pandas as pd, json
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling,  DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig
from datasets import Dataset

ImportError: cannot import name 'DataCollatorForLanguageModeling' from 'transformers' (d:\anaconda\Lib\site-packages\transformers\__init__.py)

In [ ]:
import torch
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"
device = torch.device(dev)

In [ ]:
device

device(type='cpu')

# Model and Tokenization loading

In [ ]:
# We load the model
base_model = GPT2LMHeadModel.from_pretrained('gpt2')
# options: ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']


## Model and parameters architecture

In [ ]:
base_model.num_parameters
# (wte): Embedding(50262, 768)
#     (wpe): Embedding(1024, 768)


<bound method ModuleUtilsMixin.num_parameters of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)>

## Load tokenizer

In [ ]:
# # We load the tokenizer
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
base_tokenizer

GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [ ]:
print('Words in vocabulary: ', base_tokenizer.vocab_size)


Words in vocabulary:  50257


In [ ]:
vocabulary = base_tokenizer.get_vocab()
vocabulary['Hi']


17250

In [ ]:
text = "Hi, I'm Victor and I work as a Data Scientist"
base_tokenizer.tokenize(text)


['Hi',
 ',',
 'ĠI',
 "'m",
 'ĠVictor',
 'Ġand',
 'ĠI',
 'Ġwork',
 'Ġas',
 'Ġa',
 'ĠData',
 'ĠScientist']

And to prepare the text and convert it to the format expected by the model, we would use the encode function, specifying in which format we want it to generate the tensor, Pytorch or TensorFlow.

In [ ]:
text_ids = base_tokenizer.encode(text, return_tensors = 'pt')
text_ids

# tensorflow
#text_ids = base_tokenizer.encode(text, return_tensors = 'tf')


tensor([[17250,    11,   314,  1101, 12622,   290,   314,   670,   355,   257,
          6060, 33374]])

## Decoding methods and parameters

In [ ]:
text = "I work as a data scientist"
text_ids = base_tokenizer.encode(text, return_tensors = 'pt')

generated_text_samples = base_model.generate(
    text_ids
)
generated_text_samples


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
d:\Documents Personnels\Projets\Portfolio project\LLM_GPT\cuda\Lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[   40,   670,   355,   257,  1366, 11444,   379,   262,  2059,   286,
          3442,    11, 14727,    13,   198,   198,     1,    40,  1101,   407]])

As the output is again a tensor, we will have to decode the output using the tokenizer token by token:

In [ ]:
for i, beam in enumerate(generated_text_samples):
    print(f"{i}: {base_tokenizer.decode(beam, skip_special_tokens=True)}")
    print()


0: I work as a data scientist at the University of California, Berkeley.

"I'm not



In [ ]:
def generate_n_text_samples(model, tokenizer, input_text, device, n_samples = 5):
    text_ids = tokenizer.encode(input_text, return_tensors = 'pt')
    text_ids = text_ids.to(device)
    model = model.to(device)

    generated_text_samples = model.generate(
        text_ids, 
        max_length= 100,  
        num_return_sequences= n_samples,
        no_repeat_ngram_size= 2,
        repetition_penalty= 1.5,
        top_p= 0.92,
        temperature= .85,
        do_sample= True,
        top_k= 125,
        early_stopping= True
    )
    gen_text = []
    for t in generated_text_samples:
        text = tokenizer.decode(t, skip_special_tokens=True)
        gen_text.append(text)

        return gen_text


# Fine-Tunning for fake news generator

In [ ]:
# special tokens are defined
bos = '<|endoftext|>'
eos = '<|EOS|>'
body = '<|body|>'
additional_special_tokens = [body]

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': '<pad>',
                       'sep_token': body} 
                      #  'additional_special_tokens':additional_special_tokens}

# the new token is added to the tokenizer
num_added_toks = base_tokenizer.add_special_tokens(special_tokens_dict)

# model configuration to which we add the special tokens
config = AutoConfig.from_pretrained('gpt2', 
                                    bos_token_id=base_tokenizer.bos_token_id,
                                    eos_token_id=base_tokenizer.eos_token_id,
                                    pad_token_id=base_tokenizer.pad_token_id,
                                    sep_token_id=base_tokenizer.sep_token_id,
                                    output_hidden_states=False)

# we load the pre-trained model with custom settings
base_model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)

# model embeding resizing
base_model.resize_token_embeddings(len(base_tokenizer))


Embedding(50260, 768)

## Load and preprocessing dataset

In [ ]:
# filepath= './dataset/articles1.csv'
# df = pd.read_csv(filepath, encoding = 'utf-8', usecols=['title', 'publication'])\
#                     .rename(columns={'title': 'text'})

# pd.set_option("display.max_colwidth", None)
# df.head(5)


In [ ]:
df = []
for filepath in ['./dataset/articles1.csv', './dataset/articles2.csv']:
    news_df = pd.read_csv(filepath, encoding = 'utf-8')
    df.append(news_df)
news_df = pd.concat(df, axis=0)

def remove_publication_headline(headline, publication):
    # publication col doesn't match exactly with newspaper in title col
    if str(publication) in str(headline):
        headline = headline.split(' - ')[0]
    return headline

  
def process_headlines_articles(df, title_col, content_col):
    # Remove rows with empty or null title or content
    titulo_vacio = (df[title_col].str.len() == 0) | df[title_col].isna()
    contenido_vacio = (news_df[content_col].str.len() == 0) | news_df[content_col].isna()
    df = df[~titulo_vacio & ~contenido_vacio]

    # # Remove publication name from title
    df[title_col] = df.apply(lambda row: remove_publication_headline(row[title_col], row['publication']), axis = 1)

    # # Remove headlines with less than 8 words
    titlos_len_ge8 = (df[title_col].str.split().apply(lambda x: len(x)) >= 8)
    df = df[titlos_len_ge8]

    # # Keep the first 100 words from the content
    news_df[content_col] = news_df[content_col].str.split(' ').apply(lambda x: ' '.join(x[:100]))

    # # Drop duplicates
    text_df = df.drop_duplicates(subset = [title_col, content_col])\
                [[title_col, content_col]]

    return text_df

# Data cleansing
news_df = process_headlines_articles(news_df, title_col='title', content_col='content')

# We add the tokens
prepare_text = lambda x: ' '.join([bos, x['title'], body, x['content'], eos])
news_df['text'] = news_df.apply(prepare_text, axis=1)

# Split in train and test
df_train_news, df_val_news = train_test_split(news_df, train_size = 0.9, random_state = 77)
print(f'There are {len(df_train_news)} headlines for training and {len(df_val_news)} for validation')

# we load the datasets from pandas df
train_dataset = Dataset.from_pandas(df_train_news[['text']])
val_dataset = Dataset.from_pandas(df_val_news[['text']])




C:\Users\audre\AppData\Local\Temp\ipykernel_11348\3635246350.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[title_col] = df.apply(lambda row: remove_publication_headline(row[title_col], row['publication']), axis = 1)


There are 69768 headlines for training and 7753 for validation


In [ ]:
train_dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 69768
})

## Tokenize the train and validation dataset

In [ ]:
max_length = base_tokenizer.model_max_length

In [ ]:
max_length

1024

In [ ]:
def tokenize_function(examples):
    text = base_tokenizer(examples['text'], truncation=True, max_length=1024)
    if len(text) > 1024:
        print(len(text))
        text = text[:1024]
    return text

In [ ]:
# tokenization
tokenized_train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=1
)

Map:   0%|          | 0/69768 [00:00<?, ? examples/s]

In [ ]:
#tokenization
tokenized_val_dataset = val_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=1
)

Map:   0%|          | 0/7753 [00:00<?, ? examples/s]

# Training

In [ ]:
model_articles_path = './news-articles_v4'

training_args = TrainingArguments(
    output_dir=model_articles_path,          # output directory
    num_train_epochs=2,              # total # of training epochs
    per_device_train_batch_size=5,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=model_articles_path,            # directory for storing logs
    prediction_loss_only=True,
    save_steps=10000
)

data_collator = DataCollatorForLanguageModeling(
        tokenizer=base_tokenizer,
        mlm=False
    )

trainer = Trainer(
    model=base_model,                         # the instantiated  Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,         # training dataset
    eval_dataset=tokenized_val_dataset,            # evaluation dataset
    
)


In [ ]:
trainer.train()


NameError: name 'trainer' is not defined

In [ ]:
trainer.train(resume_from_checkpoint=True)


In [ ]:
trainer.save_model()
base_tokenizer.save_pretrained(model_path)


In [ ]:
# trained model loading
model_articles_path = './news-articles_v4'

news_model = GPT2LMHeadModel.from_pretrained(model_articles_path)
news_tokenizer = GPT2Tokenizer.from_pretrained(model_articles_path)

bos = news_tokenizer.bos_token
eos = news_tokenizer.eos_token
sep = news_tokenizer.additional_special_tokens[0]

articles = {}
headline = "crisis"
content = generate_n_text_samples(news_model, news_tokenizer, headline, 
                                      device, n_samples = 1)[0]
articles = content.replace(headline, '')

for title, content in articles.items():
    print('\033[1m' + title + '\033[0m')
    pretty_print(content)
    print()
